In [ ]:
# CONFIG & PARAMS
import pandas as pd
from pathlib import Path
from datetime import datetime

INPUT_PATH = Path("ventas.csv")
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# parámetros reusables
FECHA_DESDE = "2025-07-01"
FECHA_HASTA = "2025-07-31"


In [ ]:
# EXTRACT ─ cargar datos crudos
df = pd.read_csv(INPUT_PATH)
display(df.head())
print(df.dtypes)
print("Filas crudas:", len(df))


In [ ]:
# CLEAN ─ normalizar nombres y tipos
df.columns = (df.columns
                .str.strip().str.lower().str.replace(" ", "_"))

# tipado seguro
for c in ("precio", "cantidad"):
    df[c] = pd.to_numeric(df[c], errors="coerce")

# limpieza básica
df["categoria"] = df["categoria"].astype(str).str.strip().str.title()

# validar nulos críticos
nulls = df[["precio","cantidad"]].isna().sum().sum()
print("Nulos en precio/cantidad:", nulls)
df = df.dropna(subset=["precio","cantidad"])
print("Filas post-clean:", len(df))


In [ ]:
# TRANSFORM ─ crear métricas
df["ingresos"] = df["precio"] * df["cantidad"]

# filtro temporal parametrizado
mask = df["fecha"].between(FECHA_DESDE, FECHA_HASTA)
df_mes = df.loc[mask].copy()

print("Filas periodo:", len(df_mes))


In [ ]:
# AGG ─ resumen por categoría
reporte = (df_mes
           .groupby("categoria", dropna=False)["ingresos"]
           .sum()
           .reset_index()
           .sort_values("ingresos", ascending=False))
display(reporte.head(10))


In [ ]:
# VALIDATIONS ─ garantías básicas
assert (reporte["ingresos"] >= 0).all(), "Ingresos negativos detectados"
assert not reporte["categoria"].isna().any(), "Categorías nulas en el reporte"
print("Validaciones OK ✅")


In [ ]:
# LOAD ─ persistir salida versionada
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_path = OUTPUT_DIR / f"reporte_categorias_{FECHA_DESDE}_{FECHA_HASTA}_{ts}.csv"
reporte.to_csv(out_path, index=False)
print("Guardado:", out_path)
